In [1]:
import sqlite3
db = sqlite3.connect("APAD_proj.db")

In [2]:
def email_check(db,email):
    cursor = db.cursor()
    while(1):
        findEmail = '''SELECT * FROM user where email_id = ?'''
        cursor.execute(findEmail, (email,))
        if cursor.fetchall():
            print("Email already registered, enter again")
            email = str(input("Please provide your email-address: "))
        else:
            return email


In [3]:
def admin_check(db,email):
    cursor = db.cursor()
    findEmail = '''SELECT email_id FROM user where first_name = ?'''
    cursor.execute(findEmail, ("admin",))
    admin = cursor.fetchone()[0]
#     print(admin)
    while email != admin:
        print("Only admin is authorized to add a venue")
        email = str(input("Enter the admin's email address: "))
    
    return email



In [4]:
def password_verify(pwd):
    while pwd[0]!=pwd[1]:
        print("Your passwords didn't match, enter again")
        pwd[0] = str(input("Enter the password: ")) # NEED to add constraints
        pwd[1] = str(input("Enter the password again to recheck: "))
    return pwd[0], pwd[1]
        

In [5]:
def new_user(db, inputs):

    cursor = db.cursor()
    query = '''INSERT INTO user(email_id,first_name,last_name,contact_number, zip_code, password)
                    VALUES(?,?,?,?,?,?)'''

    cursor.execute(query, (inputs[:6]))  
    db.commit()


In [6]:
def user_input_form():
    email = str(input("Please provide your email-address: "))
    email = email_check(db, email)
    first_name = str(input("Please Enter your first name: "))
    last_name = str(input("Please Enter your last name: "))
    contact_number = str(input("Please provide a valid contact number: "))
    zip_code = str(input("Your zip code: "))
    password = str(input("Enter the password: ")) # NEED to add constraints
    password_check = str(input("Enter the password again to recheck: "))
    pwd = [password, password_check]
    password, password_check = password_verify(pwd)
    inputs = [email, first_name, last_name, contact_number, zip_code, password, password_check]
    
    return inputs

In [ ]:
# Single User input

inputs = user_input_form()
new_user(db, inputs)

In [7]:
def new_venue(db, inputs):
    
    cursor = db.cursor()
    
    query = '''INSERT INTO venue(venue_name, address, zip_code, contact_number, description, open_time, close_time, games_total_count, games_available_count)
                VALUES(?,?,?,?,?,?,?,?,?)'''

    # cursor = db.cursor()
    cursor.execute(query, inputs_venue[1:]) 
    db.commit()


In [8]:
def add_venue_form():
    email = str(input("Please enter your email address: "))
    email1 = admin_check(db,email)
    venue_name = str(input("Please Enter the venue name: "))
    address = str(input("Please Enter the venue address: "))
    zip_code = str(input("Venue zip code: "))
    contact_number = str(input("Please provide a contact number: "))
    description = str(input("Enter the venue description (not more than 200 characters): ")) # Need to limit the length of input
    open_time = str(input("Please Enter the start time in hrs: "))
    close_time   = str(input("Please Enter the end time in hrs: "))
    games_total_count = int(input("Please enter the total number of games that can be played in single time slot: "))
#     games_available_count = int(input("Please enter the available number of games that can be played in single time slot: "))
    games_available_count = 0
    inputs_venue = [email1,venue_name,address,zip_code,contact_number,description,open_time,close_time,games_total_count, games_available_count]
    return inputs_venue

In [ ]:
inputs_venue = add_venue_form()
new_venue(db, inputs_venue)

In [9]:
def email_verify(db, email):
    cursor = db.cursor()
    
    findEmail = '''SELECT email_id FROM user where email_id = ?'''
    cursor.execute(findEmail, (email,))
    
    if cursor.fetchall():
        return email
    else:
        print("You can't create an event yet. Create an account first")
        inputs = user_input_form()
        new_user(db, inputs)
        return inputs[0]

In [10]:
def fetch_event_cat(cursor):
    event_cat_query = '''SELECT event_category_name FROM event_category'''
    cursor.execute(event_cat_query)
    event_cat = cursor.fetchall()
    print("Please select a category from the following: ") 
    for i in range(len(event_cat)):
        print(str(i) +":" + event_cat[i])
    
    

In [11]:
def fetch_venue(cursor):
    venue_query = '''SELECT venue_name FROM venue'''
    cursor.execute(venue_query)
    venues = cursor.fetchall()
    print("Please select a venue from the following: ") 
    for i in range(len(venues)):
        print(str(i) +":" + venues[i])
    return venues

In [12]:
def fetch_sport(cursor):
    sport_query = '''SELECT sport_name FROM sports_cat'''
    cursor.execute(sport_query)
    sports = cursor.fetchall()
    print("Please select a sport from the following: ") 
    for i in range(len(sports)):
        print(str(i) +":" + sports[i])
    return sports

In [13]:
def new_event(db,inputs):
    
    cursor = db.cursor()

    if inputs[9] == "create":

        host_insert_flag = '''INSERT INTO events(event_cat_id,venue_id,event_name, date, start_time,end_time,
                                              user_id, host_flag, member_flag, sports_cat_id, event_desc, capacity_avail)
                       VALUES(?,?,?,?,?,?,?,?,?,?,?,?)'''


        cursor.execute(host_insert_flag, (inputs[7],inputs[8],inputs[1],inputs[2],inputs[3],inputs[4],inputs[0],1,0,inputs[10],inputs[5], inputs[6]))  

        db.commit()
        
        # -1 the count of games_available count in venue table
        
        update_game_avail_ct  = '''UPDATE venue SET games_available_count = ? WHERE venue_id = ?'''
        cursor.execute(update_game_avail_ct,(inputs[11]-1,inputs[8]))
        db.commit()


    elif inputs[9] == "join":

        find_event = '''SELECT * FROM events where date = ?
                            and start_time = ? and end_time = ?'''
        cursor.execute(find_event, (inputs[2], inputs[3], inputs[4],))
        p = cursor.fetchall()
    else:
        print("enter a valid option")

In [14]:
def event_create_inputs(db):
    
    count = 0
    cursor = db.cursor()
    db.row_factory = lambda cursor, row: row[0]
    #Input the email of the user
    email = str(input("Please provide your email-address: "))

    #Check if it's an existing user
    email = email_verify(db,email)
    
    create_or_join = str(input("Do you want to create/join the game: ")).lower()
    
    #Get the user_id from the database
    user_id_query = '''SELECT user_id FROM user where email_id = ?'''
    cursor.execute(user_id_query, (email,))
    user_id = cursor.fetchone()
    
    event_name = str(input("Please Enter the event name: "))
    
    #Fetch the list of sports 
    sports = fetch_sport(cursor)
    
    #Input for sport. Only accept if it exists!
    while(1):
        sport_name = str(input("Please Enter the sport name: "))
        for i in sports:
            if (sport_name == i):
                count=1
                break
        if count==1:
            break
        else:
            print("Please enter the correct sport from choices above!")

    #Get the sport id from the database
    sports_cat_id_query = '''SELECT sports_cat_id FROM sports_cat where sport_name = ?'''
    cursor.execute(sports_cat_id_query, (sport_name,))
    sports_cat_id = cursor.fetchone()
    
    event_date = str(input("Please Enter the event date: "))
    start_time = str(input("Please Enter the start time in hrs: "))
    end_time   = str(input("Please Enter the end time in hrs: "))
    event_desc = str(input("Please Enter the event description: "))
    capacity   = str(input("Please Enter the capacity avail: "))
    
    #Fetch all event categories
    fetch_event_cat(cursor)
    
    #Input for event category. Only accept if it exists!
    while(1):
        event_category_name = str(input("Please Enter the event category name : "))
        if (event_category_name == "watch" or event_category_name == "play" or event_category_name == "workshop" ):
            break
        else:
            print("Please enter the right category from choices above!")
    
    #Get the Event cat id from the database
    event_cat_id_query = '''SELECT event_cat_id FROM event_category where event_category_name = ?'''
    cursor.execute(event_cat_id_query, (event_category_name,))
    event_cat_id = cursor.fetchone()
    
    #Fetch all the venues
    venues = fetch_venue(cursor)
    
    #Input for venue. Only accept if it exists!
    count=0
    while(1):
        venue_name = str(input("Please Enter the venue name: "))
        for i in venues:
            if (venue_name == i):
                count=1
                break
        if count==1:
            break
        else:
            print("Please enter the correct venue from choices above!")
    
    #Get the venue id from the database
    venue_id_query = '''SELECT venue_id FROM venue where venue_name = ?'''
    cursor.execute(venue_id_query, (venue_name,))
    venue_id = cursor.fetchone()
    
    #Get the available no. of game counts from a venue
    venue_count_query = '''SELECT games_available_count FROM venue where venue_id = ?'''
    cursor.execute(venue_count_query, (venue_id,))
    games_avail_count = cursor.fetchone()
    
    inputs = [user_id,event_name,event_date,start_time,end_time,event_desc,capacity,event_cat_id,venue_id,create_or_join,sports_cat_id, games_avail_count]
    
    return inputs

In [ ]:
cursor = db.cursor()
db.row_factory = lambda cursor, row: row[0]

In [ ]:
inputs2 = event_create_inputs(db)
if inputs2:
    new_event(db,inputs2)

In [16]:
import datetime

In [15]:
cursor=db.cursor()

In [17]:
def fetch_venue_time(cursor):
    
    venue_avail_query = '''SELECT open_time, close_time FROM venue where venue_name = ?'''
    cursor.execute(venue_avail_query, ("Zilker Park",) )
    venues = cursor.fetchall()
#     print("Please select a venue from the following: ") 
#     for i in range(len(venues)):
#         print(str(i) +":" + venues[i])
    return venues

In [18]:
def fetch_event_time(cursor):
    
    event_avail_query = '''SELECT start_time, end_time FROM venue, events where venue.venue_id = events.venue_id AND event_name = ? OR event_name = ?'''
    cursor.execute(event_avail_query, ("Tennis at Zilker","Cricket") )
    events = cursor.fetchall()
#     print("Please select a venue from the following: ") 
#     for i in range(len(venues)):
#         print(str(i) +":" + venues[i])
    return events

In [19]:
def fetch_avail_slot(cursor):
    
    venue_avail_query = '''SELECT games_total_count, games_available_count FROM venue where venue_name = ?'''
    cursor.execute(venue_avail_query, ("Zilker Park",) )
    count = cursor.fetchone()
#     print("Please select a venue from the following: ") 
#     for i in range(len(venues)):
#         print(str(i) +":" + venues[i])
    return count

In [20]:
venues = fetch_venue_time(cursor)

In [21]:
venues

[(7, 20)]

In [22]:
fetch_event_time(cursor)

[(13, 18), (10, 14)]

In [23]:
games_avail_count = fetch_avail_slot(cursor)
games_avail_count

(4, 1)

In [24]:
booked_slots = []

event_date = datetime.datetime.strptime("2019-07-27", '%Y-%m-%d').date()

print(event_date)
open_time = str(venues[0][0])
close_time = str(venues[0][1])

open_time = datetime.datetime.strptime(open_time, '%H').time()
close_time = datetime.datetime.strptime(close_time, '%H').time()

event_times = fetch_event_time(cursor)

for i in event_times:
    start_time = str(i[0]) 
    end_time = str(i[1])
    
#     print(start_time, end_time)
#     print(event_date)

    start_time = datetime.datetime.strptime(start_time, '%H').time()
    end_time = datetime.datetime.strptime(end_time, '%H').time()
#     print(start_time, end_time)
#     print("Hey")
#     booked_slots= [((datetime.datetime.combine(event_date,start_time)), (datetime.datetime.combine(event_date,end_time)))]
#     print(booked_slots)
#     print(datetime.datetime.combine(event_date,start_time).date()
    booked_slots.append(((datetime.datetime.combine(event_date,start_time)), (datetime.datetime.combine(event_date,end_time))))
    print("1")
booked_slots

2019-07-27
1
1


[(datetime.datetime(2019, 7, 27, 13, 0),
  datetime.datetime(2019, 7, 27, 18, 0)),
 (datetime.datetime(2019, 7, 27, 10, 0),
  datetime.datetime(2019, 7, 27, 14, 0))]

In [25]:
start_time = '9:00'
end_time = '18:00'
time = datetime.datetime.strptime(start_time, '%H:%M')
end = datetime.datetime.strptime(end_time, '%H:%M')
hours = []

while time <= end:
    hours.append(time)
    time += datetime.timedelta(hours=1)
    
hours

[datetime.datetime(1900, 1, 1, 9, 0),
 datetime.datetime(1900, 1, 1, 10, 0),
 datetime.datetime(1900, 1, 1, 11, 0),
 datetime.datetime(1900, 1, 1, 12, 0),
 datetime.datetime(1900, 1, 1, 13, 0),
 datetime.datetime(1900, 1, 1, 14, 0),
 datetime.datetime(1900, 1, 1, 15, 0),
 datetime.datetime(1900, 1, 1, 16, 0),
 datetime.datetime(1900, 1, 1, 17, 0),
 datetime.datetime(1900, 1, 1, 18, 0)]

In [26]:
from datetime import timedelta

In [27]:
total_hours = ((datetime.datetime.combine(event_date,open_time)), (datetime.datetime.combine(event_date,close_time)))
total_hours

(datetime.datetime(2019, 7, 27, 7, 0), datetime.datetime(2019, 7, 27, 20, 0))

In [28]:
booked_slots

[(datetime.datetime(2019, 7, 27, 13, 0),
  datetime.datetime(2019, 7, 27, 18, 0)),
 (datetime.datetime(2019, 7, 27, 10, 0),
  datetime.datetime(2019, 7, 27, 14, 0))]

In [34]:
def count(listOfTuple): 
      
    count_map = {} 
    for i in listOfTuple: 
        count_map[i] = count_map.get(i, 0) +1
#     print(count_map)
    return count_map

In [30]:
duration=timedelta(hours=1)
slots = sorted([(total_hours[0],total_hours[0])]+ booked_slots + [(total_hours[1],total_hours[1])])
print(slots)

[(datetime.datetime(2019, 7, 27, 7, 0), datetime.datetime(2019, 7, 27, 7, 0)), (datetime.datetime(2019, 7, 27, 10, 0), datetime.datetime(2019, 7, 27, 14, 0)), (datetime.datetime(2019, 7, 27, 13, 0), datetime.datetime(2019, 7, 27, 18, 0)), (datetime.datetime(2019, 7, 27, 20, 0), datetime.datetime(2019, 7, 27, 20, 0))]


In [35]:
print("-----Time Slots Avalibility at Zilker----\n")
for start, end in ((slots[i][1], slots[i+1][0]) for i in range(len(slots)-1)):
#     assert start <= end, "Cannot attend all appointments"
    while start + duration <= end:
        if games_avail_count!=0:
            print ("{:%H:%M} - {:%H:%M}".format(start, start + duration) + "\t" + "(" + str(games_avail_count[0]) + ")")
            start += duration
hours = []    
for time,end in booked_slots:   
    
    while time <= end:
        hours.append(time)
        time += datetime.timedelta(hours=1)
    
hours = (sorted(hours))    
# print(hours)
temp = []
i=0
while(i < len(hours)-1):
    if hours[i]!=hours[i+1]:
        temp.append((hours[i],hours[i+1]))
        i+=1
    else:
        temp.append((hours[i],hours[i+2]))
        temp.append((hours[i+1],hours[i+3]))
        i+=3
#     print(i)

temp1 = count(temp)
for items in temp1:
    print("{:%H:%M} - {:%H:%M}".format(items[0],items[1]) + "\t" + "(" + str(games_avail_count[0]-temp1[items]) + ")")    
    
#   print ("{:%H:%M} - {:%H:%M}".format(i,j) + "\t" + "(" + str(games_avail_count[1]) + ")")

-----Time Slots Avalibility at Zilker----

07:00 - 08:00	(4)
08:00 - 09:00	(4)
09:00 - 10:00	(4)
18:00 - 19:00	(4)
19:00 - 20:00	(4)
10:00 - 11:00	(3)
11:00 - 12:00	(3)
12:00 - 13:00	(3)
13:00 - 14:00	(2)
14:00 - 15:00	(3)
15:00 - 16:00	(3)
16:00 - 17:00	(3)
17:00 - 18:00	(3)


In [ ]:
temp1 = count(temp)

for items in temp1:
    print("{:%H:%M} - {:%H:%M}".format(items[0],items[1]) + "\t" + "(" + str(games_avail_count[0]-temp1[items]) + ")")

In [ ]:
from datetime import datetime

In [ ]:
event_date = datetime.date(2019)
start_time = datetime.time(10)

datetime.datetime.combine(event_date,start_time)

In [ ]:
datetime.datetime.strptime('01', '%H').time()

In [ ]:
db.close()

In [ ]:
open_time = str(venues[0][0])
close_time = str(venues[0][1])

open_time = datetime.datetime.strptime(open_time, '%H').time()
close_time = datetime.datetime.strptime(close_time, '%H').time()
open_time

In [ ]:
total_hours = ((datetime.datetime.combine(event_date,open_time)), (datetime.datetime.combine(event_date,close_time)))

In [ ]:
total_hours